# DeepDoubleTau

## Setup

In [4]:
import os,sys
os.environ['KERAS_BACKEND'] = 'tensorflow'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import tensorflow as tf
#import keras
import numpy as np
#from keras import backend as K
#from tensorflow.python.client import device_lib
#device_lib.list_local_devices()
from optparse import OptionParser
import pandas as pd
import h5py
import json
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam, Nadam, SGD
from tensorflow.keras.utils import to_categorical
import matplotlib
matplotlib.use('agg')
#%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, concatenate, BatchNormalization, GRU
from tensorflow.keras.models import Model 
from tensorflow.keras.regularizers import l1
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import roc_curve, auc, roc_auc_score
from tensorflow.keras.models import model_from_json
import yaml
from scipy.interpolate import griddata
import yaml

fColors = {
'black'    : (0.000, 0.000, 0.000), # hex:000000
'blue'     : (0.122, 0.467, 0.706), # hex:1f77b4
'orange'   : (1.000, 0.498, 0.055), # hex:ff7f0e
'green'    : (0.173, 0.627, 0.173), # hex:2ca02c
'red'      : (0.839, 0.153, 0.157), # hex:d62728
'purple'   : (0.580, 0.404, 0.741), # hex:9467bd
'brown'    : (0.549, 0.337, 0.294), # hex:8c564b
'darkgrey' : (0.498, 0.498, 0.498), # hex:7f7f7f
'olive'    : (0.737, 0.741, 0.133), # hex:bcbd22
'cyan'     : (0.090, 0.745, 0.812)  # hex:17becf
}

colorlist = ['blue','orange','green','red','purple','brown','darkgrey','cyan','deepskyblue','dimgrey','lightcoral','teal','darkviolet','magenta','chocolate','dodgerblue','olive','lawngreen','paleturquoise','darkred']

with open("../pf.json") as jsonfile:
    payload = json.load(jsonfile)
    weight = payload['weight']
    features = payload['features']
    altfeatures = payload['altfeatures']
    cut = payload['cut']
    ss = payload['ss_vars']
    label = payload['!decayType']

# columns declared in file
lColumns = weight + ss
nparts = 30
lPartfeatures = []
for iVar in features:
    for i0 in range(nparts):
        lPartfeatures.append(iVar+str(i0))
nsvs = 5
lSVfeatures = []
for iVar in altfeatures:
    for i0 in range(nsvs):
        lSVfeatures.append(iVar+str(i0))
lColumns = lColumns + lPartfeatures + lSVfeatures + [label]

features_to_plot = weight + ss

fill_factor = 5

feature_range = {
        "fj_msd":[40.,215.],
        "fj_pt":[300.,700.],
        }

bin_dict = {
        "fj_pt":np.arange(300.,825.,25.),
        "fj_msd":np.arange(40.,215.,5.),
        }

mincut = -1.
maxcut = 1.
stepsize = 0.05
ddt_pts = [0.01,0.05,0.1,0.5]

### If you want to remake input files (not necessary)

In [ ]:
def remake(iFiles_sig, iFiles_bkg, iFile_out):


    features_labels_df_sig = pd.DataFrame(columns=lColumns)

    for sig in iFiles_sig:
        h5File_sig = h5py.File(sig)
        treeArray_sig = h5File_sig['deepDoubleTau'][()]
        print(treeArray_sig.shape)
        tmp_sig = pd.DataFrame(treeArray_sig,columns=lColumns)
        features_labels_df_sig = pd.concat([features_labels_df_sig,tmp_sig])

    sighist,_x,_y = np.histogram2d(features_labels_df_sig[weight[0]],features_labels_df_sig[weight[1]],bins=20,range=np.array([[300.,800.],[40.,240.]]))
    print(np.sum(sighist))

    remade_df_bkg = pd.DataFrame(columns=lColumns)
    for bkg in iFiles_bkg:
        h5File_bkg = h5py.File(bkg)
        treeArray_bkg = h5File_bkg['deepDoubleTau'][()]
        print(treeArray_bkg.shape)
        tmp_bkg = pd.DataFrame(treeArray_bkg,columns=lColumns)

        for ix in range(len(_x)-1):
            for iy in range(len(_y)-1):
                remade_df_bkg = pd.concat([remade_df_bkg,tmp_bkg[((tmp_bkg[weight[0]] >= _x[ix]) & (tmp_bkg[weight[0]] < _x[ix+1]) & (tmp_bkg[weight[1]] >= _y[iy]) & (tmp_bkg[weight[1]] < _y[iy+1]))].head(int(sighist[ix,iy])*fill_factor)], ignore_index = True)

    bkghist,_,_ = np.histogram2d(remade_df_bkg[weight[0]],remade_df_bkg[weight[1]],bins=20,range=np.array([[300.,800.],[40.,240.]]))
    print(np.nan_to_num(np.divide(bkghist,sighist)))

    arr = np.concatenate([features_labels_df_sig.values,remade_df_bkg.values],axis=0)
    print(arr.shape)
    # open HDF5 file and write dataset
    h5File = h5py.File(iFile_out,'w')
    h5File.create_dataset('deepDoubleTau', data=arr,  compression='lzf')
    h5File.close()
    del h5File

remake(['../GluGluHToTauTau_user_hadel.z'],['../QCD.z','../TTbar.z'],'../comb_distcut%i_hadel.z'%fill_factor)
remake(['../GluGluHToTauTau_user_hadmu.z'],['../QCD.z','../TTbar.z'],'../comb_distcut%i_hadmu.z'%fill_factor)
remake(['../GluGluHToTauTau_user_hadhad.z'],['../QCD.z','../TTbar.z'],'../comb_distcut%i_hadhad.z'%fill_factor)

### Function definitions

In [2]:
def load_comb(iFile, iFileNpy='',iNparts=20,iNSVs=5):
    h5File = h5py.File(iFile)
    treeArray = h5File['deepDoubleTau'][()]
    print(treeArray.shape)

    features_labels_df = pd.DataFrame(treeArray,columns=lColumns)

    idconv = {211.:1, 13.:2,  22.:3,  11.:4, 130.:5, 1.:6, 2.:7, 3.:8, 4.:9,
            5.:10, -211.:1, -13.:2,
            -11.:4, -1.:-6, -2.:7, -3.:8, -4.:9, -5.:10, 0.:0}
    nIDs = 33
    for i0 in range(nparts):
        features_labels_df['PF_id'+str(i0)] = features_labels_df['PF_id'+str(i0)].map(idconv)
    selPartfeatures = []
    for iVar in features:
        for i0 in range(iNparts):
            selPartfeatures.append(iVar+str(i0))
    selSVfeatures = []
    for iVar in altfeatures:
        for i0 in range(iNSVs):
            selSVfeatures.append(iVar+str(i0))
    features_df        = features_labels_df[selPartfeatures].values
    features_sv_df        = features_labels_df[selSVfeatures].values
    labels          = features_labels_df[label]
    #features_val       = features_df.values
    feat_val           = features_labels_df[features_to_plot].values

    for p in selPartfeatures:
        if (features_labels_df[p].isna().sum()>0): print(p,"found nan!!")

    #if iFileNpy!='':
    #    features_2df = np.load(iFileNpy)
    #else:
    print(features_df.shape)
    features_df = features_df.reshape(-1,iNparts,len(features))
    print(features_df.shape)
    print(features_sv_df.shape)
    features_sv_df = features_sv_df.reshape(-1,iNSVs,len(altfeatures))
    print(features_sv_df.shape)
    features_val = features_df
    features_sv_val = features_sv_df
    labels_val = labels
    feat_val = feat_val

    print(features_val)
    # split into random test and train subsets 
    X_train_val, X_test, Xalt_train_val, Xalt_test, y_train_val, y_test, feat_train, feat_test = train_test_split(features_val, features_sv_val, labels_val, feat_val, test_size=0.2, random_state=42)
    #scaler = preprocessing.StandardScaler().fit(X_train_val)
    #X_train_val = scaler.transform(X_train_val)
    #X_test      = scaler.transform(X_test)
    return X_train_val, X_test, Xalt_train_val, Xalt_test, y_train_val, y_test, feat_train, feat_test


def load(iFile_sig, iFile_bkg, iFileNpy='',iNparts=20,iNSVs=5):
    h5File_sig = h5py.File(iFile_sig)
    h5File_bkg = h5py.File(iFile_bkg)
    treeArray_sig = h5File_sig['deepDoubleTau'][()]
    treeArray_bkg = h5File_bkg['deepDoubleTau'][()]
    #treeArray_bkg = treeArray_bkg[:10000,:]
    treeArray_bkg = treeArray_bkg[:1500000,:]
    print(treeArray_sig.shape)
    print(treeArray_bkg.shape)

    features_labels_df_sig = pd.DataFrame(treeArray_sig,columns=lColumns)
    features_labels_df_bkg = pd.DataFrame(treeArray_bkg,columns=lColumns)

    idconv = {211.:1, 13.:2,  22.:3,  11.:4, 130.:5, 1.:6, 2.:7, 3.:8, 4.:9,
            5.:10, -211.:1, -13.:2,
            -11.:4, -1.:-6, -2.:7, -3.:8, -4.:9, -5.:10, 0.:0}
    nIDs = 33
    for i0 in range(nparts):
        features_labels_df_sig['PF_id'+str(i0)] = features_labels_df_sig['PF_id'+str(i0)].map(idconv)
        features_labels_df_bkg['PF_id'+str(i0)] = features_labels_df_bkg['PF_id'+str(i0)].map(idconv)
    selPartfeatures = []
    for iVar in features:
        for i0 in range(iNparts):
            selPartfeatures.append(iVar+str(i0))
    selSVfeatures = []
    for iVar in altfeatures:
        for i0 in range(iNSVs):
            selSVfeatures.append(iVar+str(i0))
    print(features_labels_df_sig[label])
    print(features_labels_df_bkg[label])
    features_df_sig        = features_labels_df_sig[selPartfeatures].values
    features_df_bkg        = features_labels_df_bkg[selPartfeatures].values
    features_sv_df_sig        = features_labels_df_sig[selSVfeatures].values
    features_sv_df_bkg        = features_labels_df_bkg[selSVfeatures].values
    labels_sig        = features_labels_df_sig[label]
    labels_bkg        = features_labels_df_bkg[label]
    #features_val       = features_df.values
    feat_val_sig           = features_labels_df_sig[features_to_plot].values
    feat_val_bkg           = features_labels_df_bkg[features_to_plot].values

    for p in selPartfeatures:
        if (features_labels_df_sig[p].isna().sum()>0): print(p,"found nan!!")
        if (features_labels_df_bkg[p].isna().sum()>0): print(p,"found nan!!")

    #if iFileNpy!='':
    #    features_2df = np.load(iFileNpy)
    #else:
    print(features_df_sig.shape)
    features_df_sig = features_df_sig.reshape(-1,iNparts,len(features))
    features_df_bkg = features_df_bkg.reshape(-1,iNparts,len(features))
    print(features_df_sig.shape)
    print(features_sv_df_sig.shape)
    features_sv_df_sig = features_sv_df_sig.reshape(-1,iNSVs,len(altfeatures))
    features_sv_df_bkg = features_sv_df_bkg.reshape(-1,iNSVs,len(altfeatures))
    print(features_sv_df_sig.shape)
    features_val = np.concatenate([features_df_sig, features_df_bkg])
    features_alt_val = np.concatenate([features_sv_df_sig, features_sv_df_bkg])
    labels_val = np.concatenate([labels_sig, labels_bkg])
    feat_val = np.concatenate([feat_val_sig, feat_val_bkg])

    print(features_val)
    # split into random test and train subsets 
    X_train_val, X_test, Xalt_train_val, Xalt_test, y_train_val, y_test, feat_train, feat_test = train_test_split(features_val, features_alt_val, labels_val, feat_val, test_size=0.2, random_state=42)
    #scaler = preprocessing.StandardScaler().fit(X_train_val)
    #X_train_val = scaler.transform(X_train_val)
    #X_test      = scaler.transform(X_test)
    return X_train_val, X_test, Xalt_train_val, Xalt_test, y_train_val, y_test, feat_train, feat_test

def conditional_loss_function(y_true, y_pred):
    return categorical_crossentropy(y_true, y_pred)*(1-y_true[:,0])

def model(Inputs,Inputs_alt,X_train,Xalt_train,Y_train,NPARTS=20,NSV=5):
    CLR=0.01
    L1R=0.0001
    print(Inputs)
    print(Inputs_alt)

    CSTRIDE = 1
    CSIZE = int(Inputs.shape[1]-CSTRIDE*(Inputs_alt.shape[1]-1))
    print(CSIZE)

    conv = Conv1D(100,kernel_size=CSIZE,activation='relu',name='conv_base',padding='valid',activity_regularizer=l1(L1R),input_shape=Inputs.shape[1:])(Inputs)
    added = Concatenate(axis=2)([conv, Inputs_alt])

    gru_alt = GRU(100,activation='relu',recurrent_activation='hard_sigmoid',name='gru_base_alt',activity_regularizer=l1(L1R))(added)
    dense_alt   = Dense(100, activation='relu',activity_regularizer=l1(L1R))(gru_alt)
    norm_alt    = BatchNormalization(momentum=0.6, name='dense4_bnorm_alt')  (dense_alt)

    dense   = Dense(50, activation='relu',activity_regularizer=l1(L1R))(norm_alt)
    norm    = BatchNormalization(momentum=0.6, name='dense5_bnorm')  (dense)
    dense   = Dense(20, activation='relu',activity_regularizer=l1(L1R))(norm)
    dense   = Dense(10, activation='relu',activity_regularizer=l1(L1R))(dense)
    out     = Dense(1, activation='sigmoid',activity_regularizer=l1(L1R))(norm)
    
    classifier = Model(inputs=[Inputs,Inputs_alt], outputs=[out])
    lossfunction = 'binary_crossentropy'
    classifier.compile(loss=[lossfunction], optimizer=Adam(CLR), metrics=['accuracy'])
    models={'classifier' : classifier}

    return models

def model_2grus(Inputs,Inputs_alt,X_train,Xalt_train,Y_train,NPARTS=20,NSV=5):
    CLR=0.001
    L1R=0.00001
    print(Inputs)
    print(Inputs_alt)

    gru = GRU(100,activation='relu',recurrent_activation='hard_sigmoid',name='gru_base',activity_regularizer=l1(L1R))(Inputs)
    dense   = Dense(100, activation='relu',activity_regularizer=l1(L1R))(gru)
    norm    = BatchNormalization(momentum=0.6, name='dense4_bnorm')  (dense)

    gru_alt = GRU(100,activation='relu',recurrent_activation='hard_sigmoid',name='gru_base_alt',activity_regularizer=l1(L1R))(Inputs_alt)
    dense_alt   = Dense(100, activation='relu',activity_regularizer=l1(L1R))(gru_alt)
    norm_alt    = BatchNormalization(momentum=0.6, name='dense4_bnorm_alt')  (dense_alt)

    added = Add()([norm, norm_alt])

    dense   = Dense(50, activation='relu',activity_regularizer=l1(L1R))(added)
    norm    = BatchNormalization(momentum=0.6, name='dense5_bnorm')  (dense)
    dense   = Dense(20, activation='relu',activity_regularizer=l1(L1R))(norm)
    dense   = Dense(10, activation='relu',activity_regularizer=l1(L1R))(dense)
    out     = Dense(1, activation='sigmoid',activity_regularizer=l1(L1R))(norm)
    
    classifier = Model(inputs=[Inputs,Inputs_alt], outputs=[out])
    lossfunction = 'binary_crossentropy'
    classifier.compile(loss=[lossfunction], optimizer=Adam(CLR), metrics=['accuracy'])
    models={'classifier' : classifier}

    return models

def train(models,X_train,Xalt_train,Y_train,feat_train,doReweight=False):
    NEPOCHS=20
    Obatch_size=1000

    history = {}
    if (doReweight):
        sighist,_x,_y = np.histogram2d(feat_train[(Y_train.values==1),0],feat_train[(Y_train.values==1),1],bins=20,range=np.array([[300.,800.],[40.,240.]]))
        bkghist,_,_ = np.histogram2d(feat_train[(Y_train.values==0),0],feat_train[(Y_train.values==0),1],bins=[_x,_y])
        ratio_sb = np.nan_to_num(np.divide(sighist,bkghist))

        weights = np.ones(len(Y_train.values))
        binix = np.digitize(feat_train[:,0],_x)
        biniy = np.digitize(feat_train[:,1],_y)
        for i in range(len(weights)):
            if (Y_train.values[i]==0):
                if (binix[i]>0 and binix[i]<len(_x)-1 and biniy[i]>0 and biniy[i]<len(_y)-1): weights[i] = ratio_sb[binix[i],biniy[i]]

        print(weights)

        history["classifier"] = models['classifier'].fit([X_train,Xalt_train],
            Y_train,epochs=NEPOCHS,verbose=1,batch_size=Obatch_size,
            validation_split=0.2,sample_weight=weights)
    else: history["classifier"] = models['classifier'].fit([X_train,Xalt_train],
            Y_train,epochs=NEPOCHS,verbose=1,batch_size=Obatch_size,
            validation_split=0.2)
     
    return history
def plotNNResponse(data,labels,savename):
    plt.clf()
    bins=100
    for j in range(len(data)):
        plt.hist(data[j],bins,log=False,histtype='step',normed=True,label=labels[j],fill=False,range=(-1.,1.))
    plt.legend(loc='best')
    plt.xlabel('NeuralNet Response')
    plt.ylabel('Number of events (normalized)')
    plt.title('NeuralNet applied to test samples')
    plt.savefig("deepDoubleTau_%s_disc.pdf"%savename)
    plt.yscale('log')
    plt.savefig("deepDoubleTau_%s_disc_log.pdf"%savename)
    plt.yscale('linear')
    #plt.show()

def doDDT(data,feats,nbins,xbins,ybins,percents,savename):

    rho_arr = 2.*np.log(np.divide(feats[0][:,1],feats[0][:,0]))
    pt_arr = feats[0][:,0]

    xstep = (xbins[-1]-xbins[0])/float(nbins-1)
    ystep = (ybins[-1]-ybins[0])/float(nbins-1)
    binix = np.digitize(rho_arr,np.arange(xbins[0],xbins[-1]+xstep,xstep))
    biniy = np.digitize(pt_arr,np.arange(ybins[0],ybins[-1]+ystep,ystep))

    ddt_list = []
    base_list = []
    for x in range(nbins):
        for y in range(nbins):
            tmpdata = np.array(data[0])[(binix==x) & (biniy==y)]
            if (len(tmpdata)>0): ddt_list.append(np.quantile(tmpdata,1.-np.array(percents)))
            else: ddt_list.append(np.zeros(len(percents)))
            base_list.append([xbins[0]+x*xstep,ybins[0]+y*ystep])
    ddt_arr = np.array(ddt_list)
    base_grid = np.array(base_list)

    x_arr = np.array([xbins[a] for a in range(len(xbins)-1) for b in range(len(ybins)-1)])
    y_arr = np.array([ybins[b] for a in range(len(xbins)-1) for b in range(len(ybins)-1)])

    #base_grid = np.stack((np.arange(xbins[0],xbins[-1]+xstep,xstep),np.arange(ybins[0],ybins[-1]+ystep,ystep)),axis=-1)
    ddt_smooth = [griddata(base_grid, ddt_arr[:,ip], np.stack((x_arr, y_arr),axis=-1), method='cubic').clip(0.) for ip in range(len(percents))]
    print(ddt_smooth)

    for ip in range(len(percents)):
        plt.clf()
        plt.hist2d(x_arr,y_arr,weights=ddt_smooth[ip],bins=[xbins,ybins])
        plt.title('DDT Map ({}%)'.format(int(percents[ip]*100.)))
        plt.ylabel(r'$p_{T}$')
        plt.xlabel(r'$\rho = log(p_{T}^2/m_{SD}^2)$')
        plt.colorbar()
        plt.savefig("deepDoubleTau_%s_ddt_surface_%d.pdf"%(savename,int(percents[ip]*100.)))

    return ddt_smooth

def applyDDT(data,feats,xbins,ybins,themap):

    data_ddt = []
    for d in range(len(data)):

        rho_arr = 2.*np.log(np.divide(feats[d][:,1],feats[d][:,0]))
        pt_arr = feats[d][:,0]
        binix = np.digitize(rho_arr,xbins)
        biniy = np.digitize(pt_arr,ybins)
        tmpddt = np.zeros(data[d].shape)

        for x in range(len(xbins)-1):
            for y in range(len(ybins)-1):
                if (len(data[d][((binix==x) & (biniy==y))])>0): tmpddt[((binix==x) & (biniy==y))] = data[d][((binix==x) & (biniy==y))] - themap[(x*(len(ybins)-1))+y]
        data_ddt.append(tmpddt)

    return data_ddt

def plotFeatResponse(data,feats,labels, nnout_cuts, savename, ddt_grp = None):

    if (ddt_grp is not None): 
        nnout_cuts = np.zeros(len(ddt_pts))
    feats_pass = []
    for j in range(len(data)):
      bufpass = []
      for c in range(len(nnout_cuts)-1):
        tmppass = []
        tmpdata = data
        if (ddt_grp is not None):
          tmpdata = applyDDT(data,feats,ddt_grp[1],ddt_grp[2],ddt_grp[0][c])
          for x in range(len(data[j])-1):
            if (tmpdata[j][x]>0.): tmppass.append(feats[j][x])
        else:
          for x in range(len(data[j])-1):
            if (c<len(nnout_cuts)-2):
              if (tmpdata[j][x]>=nnout_cuts[c] and tmpdata[j][x]<nnout_cuts[c+1]): tmppass.append(feats[j][x])
            else:
              if (tmpdata[j][x]>=nnout_cuts[c] and tmpdata[j][x]<=nnout_cuts[c+1]): tmppass.append(feats[j][x])
        
        print("p = %i  -> %i"%(c,len(tmppass)))
        bufpass.append(tmppass)
      feats_pass.append(bufpass)
    #[channel][cuts][entries][features]

    for fi in range(len(features_to_plot)):
      plt.clf()
      nbins=25

      fig, ax = plt.subplots(1, 1)

      feat_range = [0.,1.]
      if (features_to_plot[fi] in feature_range): feat_range = feature_range[features_to_plot[fi]]

      for j in range(len(data)):
        if (j==0):
          for ci in range(len(feats_pass[j])):
            doPlot = False
            temp = np.array(feats_pass[j][ci])
            temp = temp.clip(min=0)
            temp = np.reshape(temp,(-1,len(features_to_plot)))
            pfstr = ', '
            if (ddt_grp is None): pfstr = pfstr + '{}%'.format(int(float(ci*stepsize)*100.))
            else: pfstr = pfstr + '{}%'.format(int(ddt_pts[ci]*100.))
            stylestr = "dashed"
            if (len(temp[:,fi])>0):
                ax.hist(temp[:,fi],nbins,log=False,histtype='step',normed=True,linestyle=stylestr,
                        label=labels[j]+pfstr,fill=False,range=feat_range,color=colorlist[ci])
        else:
          doPlot = False
          temp = np.array(feats[j])
          #print(temp.shape)
          temp = temp[:,fi]
          temp = temp.clip(min=0.)
          if (features_to_plot[fi] in feature_range): feat_range = feature_range[features_to_plot[fi]]
          pfstr = ''
          stylestr = "solid"
          if (len(temp)>0):
              ax.hist(temp,nbins,log=False,histtype='step',normed=True,linestyle=stylestr, 
                      label=labels[j]+pfstr,fill=False,range=feat_range,color='black')
      ax.legend(loc='best',ncol=2)
      ax.set_xlabel(features_to_plot[fi])
      ax.set_ylabel('Number of events (normalized)')
      ax.set_title('Neural Net')
      plt.savefig("deepDoubleTau_"+savename+"_"+features_to_plot[fi]+".pdf")
      plt.yscale('log')
      plt.savefig("deepDoubleTau_"+savename+"_"+features_to_plot[fi]+"_log.pdf")
      plt.yscale('linear')
      #plt.show()

def plotROC(truth, scores,labels,savename):
    plt.clf()
    for j in range(len(truth)):
        x,y,_ = roc_curve(truth[j],scores[j])
        auc = roc_auc_score(truth[j],scores[j])
        plt.plot(x,y,label='{}, AUC = {:.2f}'.format(labels[j],auc))
    plt.legend(loc='lower right')
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.savefig("deepDoubleTau_%s_roc.pdf"%savename)
    #plt.show()

def computePercentiles(data):
    
    tmp = np.quantile(data,np.arange(stepsize, 1., stepsize))
    tmpl = [mincut]
    for x in tmp:
        tmpl.append(x)
    tmpl.append(maxcut)
    return tmpl

def test(models,X_test,Xalt_test,Y_test,feat_test, savename, ddt_map=None):
    model = models['classifier']
    subsamples = [ 
                  [0,0], #bkg
                  [1,1] ] #h tautau hadhad
    labels     = ["Bkg","htt"]
    roclabels  =       ["deepDoubleTau"]
    roclabels = roclabels + ss
    response_tests   = []
    response_preds   = []
    feat_preds       = []
    roc_preds        = []
    roc_true         = []
    i0=0
    print('in test')
    for subsample in subsamples:
        print(subsample)
        ids=np.logical_and(Y_test==subsample[0],Y_test==subsample[1])
        tmpdata = [X_test[ids],Xalt_test[ids]]
        tmpfeat = feat_test[ids]
        tmppred = model.predict(tmpdata)
        print('\t',len(tmppred))
        response_tests.append(tmpdata)
        response_preds.append(tmppred)
        feat_preds.append(tmpfeat)
        if i0 > 0 and len(tmpdata[0]) > 0:
            roc_true.append([])
            roc_true[-1].extend(np.zeros(len(response_tests[0][0] )))
            roc_true[-1].extend(np.ones (len(tmpdata[0])))
            roc_preds.append([])
            roc_preds[-1].extend(response_preds[0])
            roc_preds[-1].extend(tmppred)

        for fi in range(len(weight),len(weight)+len(ss)):
            tmppred = np.array(tmpfeat)[:,fi]
            print('\t',len(tmppred))
            if i0 > 0 and len(tmpdata[0]) > 0:
                roc_true.append([])
                roc_true[-1].extend(np.zeros(len(response_tests[0][0] )))
                roc_true[-1].extend(np.ones (len(tmpdata[0])))
                roc_preds.append([])
                roc_preds[-1].extend(response_preds[0])
                roc_preds[-1].extend(tmppred)

        i0=i0+1

    print('now plotting feats')
    nnout_cuts = computePercentiles(response_preds[labels.index("Bkg")])
    print(nnout_cuts)
    print(nnout_cuts[1:-1])
    plotFeatResponse(response_preds, feat_preds, labels, nnout_cuts, savename)

    binsx = np.arange(-5.5,-1.98,0.02)
    binsy = np.arange(300.,1005.,5.)
    nbins_ddt = 10

    if (ddt_map is None):
        print('now computing/plotting DDT')
        ddt_map = doDDT(response_preds,feat_preds,nbins_ddt,binsx,binsy,np.array(ddt_pts),savename)

    #print(ddt_map)
    #for ir,r in enumerate(response_preds):
    #    print(labels[ir])
    #    print(np.histogram(r))
    print('now plotting resp')
    plotNNResponse(response_preds,labels,savename)
    print('now plotting ddt feats')
    plotFeatResponse(response_preds, feat_preds, labels, None, savename+"_ddt",[ddt_map,binsx,binsy])
    print('now plotting roc')
    plotROC(roc_true, roc_preds, roclabels, savename) 

    return ddt_map



### Do the training

In [ ]:
dtype = "hadmu"
#X_train,X_test,Y_train,Y_test,feat_train,feat_test = load('./GluGluHToTauTau_user_%s.z'%dtype,'./QCD.z')
X_train,X_test,Xalt_train,Xalt_test,Y_train,Y_test,feat_train,feat_test = load_comb('../comb_distcut5_%s.z'%dtype)

for iw,w in enumerate(weight):
    plt.clf()
    plt.hist(feat_train[(Y_train.values==0),iw],bin_dict[w],log=False,histtype='step',normed=True,fill=False,label='bkg')
    plt.hist(feat_train[(Y_train.values==1),iw],bin_dict[w],log=False,histtype='step',normed=True,fill=False,label='sig')
    plt.legend(loc='best')
    plt.xlabel(w)
    plt.ylabel('arb.')
    plt.savefig("%s_%s.pdf"%(w,dtype))
    plt.yscale('log')
    plt.savefig("%s_%s_log.pdf"%(w,dtype))
    plt.yscale('linear')

inputvars=Input(shape=X_train.shape[1:], name='input')
inputvars_alt=Input(shape=Xalt_train.shape[1:], name='altinput')
models = model(inputvars,inputvars_alt,X_train,Xalt_train,Y_train)
for m in models:
    print(str(m), models[m])
history = train(models,X_train,Xalt_train,Y_train,feat_train)
#print(len(Y_test),' vs ',sum(Y_test))
#test(models,X_test,Y_test,feat_test)
for m in models:
    model_json = models[m].to_json()
    with open("model_"+dtype+"_"+str(m)+".json", "w") as json_file:
        json_file.write(model_json)
    models[m].save_weights("model_"+dtype+"_"+str(m)+".h5")
    
    plt.clf() 
    print(history[str(m)].history.keys())
    plt.plot(history[str(m)].history['acc'])
    plt.plot(history[str(m)].history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig("%s_%s_accuracy.pdf"%(dtype,str(m)))
    plt.clf() 
    plt.plot(history[str(m)].history['loss'])
    plt.plot(history[str(m)].history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig("%s_%s_loss.pdf"%(dtype,str(m)))

### Make plots

In [5]:
dtype = 'hadmu'
X_train,X_test,Xalt_train,Xalt_test,Y_train,Y_test,feat_train,feat_test = load('../GluGluHToTauTau_user_%s.z'%dtype,'../QCD.z')
models = {}
for m in ['classifier']:
    json_file = open('model_'+dtype+'_'+str(m)+'.json', 'r')
    model_json = json_file.read()
    models[m] = model_from_json(model_json)
    models[m].load_weights("model_"+dtype+"_"+str(m)+".h5")
print("got models")
qcd_ddt = test(models,X_test,Xalt_test,Y_test,feat_test,dtype+"_QCD")
print("got ddt from QCD")
X_train,X_test,Xalt_train,Xalt_test,Y_train,Y_test,feat_train,feat_test = load('../GluGluHToTauTau_user_%s.z'%dtype,'../TTbar.z')
print("got TTbar")
test(models,X_test,Xalt_test,Y_test,feat_test,dtype+"_TTbar",ddt_map=qcd_ddt)


/home/drankin/miniconda3/envs/training/lib/python3.6/site-packages/ipykernel_launcher.py:55: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
/home/drankin/miniconda3/envs/training/lib/python3.6/site-packages/ipykernel_launcher.py:56: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.


(38443, 370)
(1500000, 370)
0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
38438    1.0
38439    1.0
38440    1.0
38441    1.0
38442    1.0
Name: fj_decayType, Length: 38443, dtype: float64
0          0.0
1          0.0
2          0.0
3          0.0
4          0.0
          ... 
1499995    0.0
1499996    0.0
1499997    0.0
1499998    0.0
1499999    0.0
Name: fj_decayType, Length: 1500000, dtype: float64
(38443, 200)
(38443, 20, 10)
(38443, 65)
(38443, 5, 13)
[[[ 4.57031250e-01  1.82373047e-01  9.52148438e-02 ...  3.87382507e-03
    3.64494324e-03  2.35939026e-03]
  [ 1.97219849e-03  1.90162659e-03  1.88636780e-03 ...  9.22679901e-05
    0.00000000e+00  0.00000000e+00]
  [-7.47070312e-02  7.89184570e-02  8.38623047e-02 ... -4.45800781e-01
   -2.95898438e-01 -7.69531250e-01]
  ...
  [ 9.96093750e-01  1.00000000e+00  9.96093750e-01 ...  1.56860352e-02
    0.00000000e+00  0.00000000e+00]
  [ 2.00000000e+00  1.00000000e+00  3.00000000e+00 ...  5.00000000e+00
 

/home/drankin/miniconda3/envs/training/lib/python3.6/site-packages/ipykernel_launcher.py:331: MatplotlibDeprecationWarning: 
The 'normed' kwarg was deprecated in Matplotlib 2.1 and will be removed in 3.1. Use 'density' instead.
/home/drankin/miniconda3/envs/training/lib/python3.6/site-packages/ipykernel_launcher.py:331: MatplotlibDeprecationWarning: 
The 'normed' kwarg was deprecated in Matplotlib 2.1 and will be removed in 3.1. Use 'density' instead.
/home/drankin/miniconda3/envs/training/lib/python3.6/site-packages/ipykernel_launcher.py:331: MatplotlibDeprecationWarning: 
The 'normed' kwarg was deprecated in Matplotlib 2.1 and will be removed in 3.1. Use 'density' instead.
/home/drankin/miniconda3/envs/training/lib/python3.6/site-packages/ipykernel_launcher.py:331: MatplotlibDeprecationWarning: 
The 'normed' kwarg was deprecated in Matplotlib 2.1 and will be removed in 3.1. Use 'density' instead.
/home/drankin/miniconda3/envs/training/lib/python3.6/site-packages/ipykernel_launcher.py

/home/drankin/miniconda3/envs/training/lib/python3.6/site-packages/ipykernel_launcher.py:331: MatplotlibDeprecationWarning: 
The 'normed' kwarg was deprecated in Matplotlib 2.1 and will be removed in 3.1. Use 'density' instead.
/home/drankin/miniconda3/envs/training/lib/python3.6/site-packages/ipykernel_launcher.py:331: MatplotlibDeprecationWarning: 
The 'normed' kwarg was deprecated in Matplotlib 2.1 and will be removed in 3.1. Use 'density' instead.
/home/drankin/miniconda3/envs/training/lib/python3.6/site-packages/ipykernel_launcher.py:331: MatplotlibDeprecationWarning: 
The 'normed' kwarg was deprecated in Matplotlib 2.1 and will be removed in 3.1. Use 'density' instead.
/home/drankin/miniconda3/envs/training/lib/python3.6/site-packages/ipykernel_launcher.py:331: MatplotlibDeprecationWarning: 
The 'normed' kwarg was deprecated in Matplotlib 2.1 and will be removed in 3.1. Use 'density' instead.
/home/drankin/miniconda3/envs/training/lib/python3.6/site-packages/ipykernel_launcher.py

now computing/plotting DDT
[array([0.00000000e+00, 5.58680254e-06, 9.77743666e-06, ...,
       5.30700912e-02, 5.09047309e-02, 4.87025172e-02]), array([0.00000000e+00, 1.57621800e-05, 2.90775111e-05, ...,
       9.68323365e-03, 9.88046217e-03, 1.00780165e-02]), array([0.00000000e+00, 7.28464354e-06, 1.37792235e-05, ...,
       4.83943999e-03, 4.97101504e-03, 5.10402852e-03]), array([0.00000000e+00, 3.61744892e-06, 6.97625070e-06, ...,
       1.38438548e-03, 1.38921541e-03, 1.39409895e-03])]
now plotting resp


...

/home/drankin/miniconda3/envs/training/lib/python3.6/site-packages/ipykernel_launcher.py:215: MatplotlibDeprecationWarning: 
The 'normed' kwarg was deprecated in Matplotlib 2.1 and will be removed in 3.1. Use 'density' instead.
/home/drankin/miniconda3/envs/training/lib/python3.6/site-packages/ipykernel_launcher.py:215: MatplotlibDeprecationWarning: 
The 'normed' kwarg was deprecated in Matplotlib 2.1 and will be removed in 3.1. Use 'density' instead.


now plotting ddt feats
p = 0  -> 11833
p = 1  -> 47418
p = 2  -> 69975
p = 0  -> 6889
p = 1  -> 6992
p = 2  -> 7008


/home/drankin/miniconda3/envs/training/lib/python3.6/site-packages/ipykernel_launcher.py:331: MatplotlibDeprecationWarning: 
The 'normed' kwarg was deprecated in Matplotlib 2.1 and will be removed in 3.1. Use 'density' instead.
/home/drankin/miniconda3/envs/training/lib/python3.6/site-packages/ipykernel_launcher.py:331: MatplotlibDeprecationWarning: 
The 'normed' kwarg was deprecated in Matplotlib 2.1 and will be removed in 3.1. Use 'density' instead.
/home/drankin/miniconda3/envs/training/lib/python3.6/site-packages/ipykernel_launcher.py:331: MatplotlibDeprecationWarning: 
The 'normed' kwarg was deprecated in Matplotlib 2.1 and will be removed in 3.1. Use 'density' instead.
/home/drankin/miniconda3/envs/training/lib/python3.6/site-packages/ipykernel_launcher.py:343: MatplotlibDeprecationWarning: 
The 'normed' kwarg was deprecated in Matplotlib 2.1 and will be removed in 3.1. Use 'density' instead.
/home/drankin/miniconda3/envs/training/lib/python3.6/site-packages/ipykernel_launcher.py

now plotting roc


[array([0.00000000e+00, 5.58680254e-06, 9.77743666e-06, ...,
        5.30700912e-02, 5.09047309e-02, 4.87025172e-02]),
 array([0.00000000e+00, 1.57621800e-05, 2.90775111e-05, ...,
        9.68323365e-03, 9.88046217e-03, 1.00780165e-02]),
 array([0.00000000e+00, 7.28464354e-06, 1.37792235e-05, ...,
        4.83943999e-03, 4.97101504e-03, 5.10402852e-03]),
 array([0.00000000e+00, 3.61744892e-06, 6.97625070e-06, ...,
        1.38438548e-03, 1.38921541e-03, 1.39409895e-03])]